# Manipulating DataFrame

## Reference
- https://spark.apache.org/docs/4.0.0/api/python/user_guide/dataprep.html

## Create DataFrame

- Example 1

      from pyspark.sql import Row

      df = spark.createDataFrame([
          Row(age=10, height=80.0, NAME="Alice"),
          Row(age=10, height=80.0, NAME="Alice"),
          Row(age=5, height=float("nan"), NAME="BOB"),
          Row(age=None, height=None, NAME="Tom"),
          Row(age=None, height=float("nan"), NAME=None),
          Row(age=9, height=78.9, NAME="josh"),
          Row(age=18, height=1802.3, NAME="bush"),
          Row(age=7, height=75.3, NAME="jerry"),
      ])

      df.show()

- Example 2

      from pyspark.sql import SparkSession
      from pyspark.sql.types import StructType, StructField, IntegerType, LongType, DoubleType, FloatType
      from pyspark.sql.types import DecimalType, StringType, BinaryType, BooleanType, DateType, TimestampType
      from decimal import Decimal
      from datetime import date, datetime

      # Define the schema of the DataFrame
      schema = StructType([
          StructField("integer_field", IntegerType(), nullable=False),
          StructField("long_field", LongType(), nullable=False),
          StructField("double_field", DoubleType(), nullable=False),
          StructField("float_field", FloatType(), nullable=False),
          StructField("decimal_field", DecimalType(10, 2), nullable=False),
          StructField("string_field", StringType(), nullable=False),
          StructField("binary_field", BinaryType(), nullable=False),
          StructField("boolean_field", BooleanType(), nullable=False),
          StructField("date_field", DateType(), nullable=False),
          StructField("timestamp_field", TimestampType(), nullable=False)
      ])

      # Sample data using the Python objects corresponding to each PySpark type
      data = [
          (123, 1234567890123456789, 12345.6789, 123.456, Decimal('12345.67'), "Hello, World!",
          b'Hello, binary world!', True, date(2020, 1, 1), datetime(2020, 1, 1, 12, 0)),
          (456, 9223372036854775807, 98765.4321, 987.654, Decimal('98765.43'), "Goodbye, World!",
          b'Goodbye, binary world!', False, date(2025, 12, 31), datetime(2025, 12, 31, 23, 59)),
          (-1, -1234567890123456789, -12345.6789, -123.456, Decimal('-12345.67'), "Negative Values",
          b'Negative binary!', False, date(1990, 1, 1), datetime(1990, 1, 1, 0, 0)),
          (0, 0, 0.0, 0.0, Decimal('0.00'), "", b'', True, date(2000, 1, 1), datetime(2000, 1, 1, 0, 0))
      ]

      # Create DataFrame
      df = spark.createDataFrame(data, schema=schema)

      # Show the DataFrame
      df.show()




In [0]:
# Load the DataFrame from previous example in PySpark-Tutorial-1-Create DataFrame
df = spark.read.csv("/Volumes/workspace/default/tutorial/BigMart Sales.csv", header=True, inferSchema=True)
df.printSchema()

# SELECT

In [0]:
# SELECT
# Form 1
df.select("Item_Identifier", "Item_Weight", "Item_Fat_Content").display()
# Form 2
df.select(df["Item_Identifier"], df["Item_Outlet_Sales"]).display()
# Form 3
from pyspark.sql import functions as sf
df.select(sf.col("Item_Identifier"), sf.col("Outlet_Location_Type")).display()


# ALIAS

In [0]:
from pyspark.sql import functions as sf
df.select(sf.col("Item_Identifier"), sf.col("Outlet_Location_Type").alias("outlet_loc_type")).display()

# Filter and where

### Use Case 1: Filter the data with fat content = Regular
### Use Case 2: Slice the data with item type = Soft Drinks and weight < 10
### Use Case 3: Fetch the data with Tier in (Tier 1 or Tier 2) and Outlet Size is null 

In [0]:
# Use Case 1: Filter the data with fat content = Regular

# df.display()
# Form 1 - df.filter()
# Form 2 - df.where()
df.filter(sf.col("Item_Fat_Content") == "Regular").display()
df.where(sf.col("Item_Fat_Content") == "Regular").display()

# ==============================================================

# Use Case 2: Slice the data with item type = Soft Drinks and weight < 10

df.filter((sf.col("Item_Type") == "Soft Drinks") & (sf.col("Item_Weight") < 10)).display()

# ==============================================================

# Use Case 3: Fetch the data with Tier in (Tier 1 or Tier 2) and Outlet Size is null

df.filter((sf.col("Outlet_Location_Type").isin("Tier 1", "Tier 2")) & (sf.col("Outlet_Size").isNull())).orderBy("Outlet_Location_Type").display()

# ==============================================================

# Rename Column(s)
- df.withColumnRenamed() -- Rename single column
- df.withColumnsRenamed() -- Rename multiple columns


In [0]:
df.withColumnRenamed("Item_Identifier", "item_id").display() # Rename single column

df.withColumnsRenamed({"Item_Identifier": "item_id", "Item_Weight": "item_weight", "Item_Fat_Content": "item_fat_content"}).display() # Rename multiple columns


# With Column function
- Create new column in the DataFrame

      Use lit() to set constant values to a new column

- Modify existing column
      
      Usage of when(), regexp_replace() with when()
      Case insensitive (?i)
      Not Null using isNotNull

In [0]:
from pyspark.sql import functions as sf

df = spark.read.csv("/Volumes/workspace/default/tutorial/BigMart Sales.csv", header=True, inferSchema=True)

df.withColumn("flag", sf.lit("new")).display() 
# Creating a new column in df with name "flag" with a constant value "new" using lit() function

df.withColumn("item_weight_mrp_product", (sf.col("Item_Weight") * sf.col("Item_MRP"))).display() 
# Creating a new column in df with name "item_weight_mrp_product" with a value by multiplying 2 columns

df.withColumn("item_fat_content_accronym", sf.when(sf.col("Item_Fat_Content") == "Low Fat", "LF")
              .when(sf.col("Item_Fat_Content") == "Regular", "REG")
              .otherwise(sf.col("Item_Fat_Content"))).display() 
# Creating a new column in df with name "item_fat_content_accronym" with a value by using when() function to map values from existing value to a new value. In this case "Low Fat" is mapped to "LF" and "Regular" is mapped to "REG".

df.withColumn(
    "Item_Fat_Content", 
    sf.when(sf.col("Item_Fat_Content").isNotNull(), 
        sf.regexp_replace(sf.col("Item_Fat_Content"), "(?i)Regular", "Reg")
    ).otherwise(sf.col("Item_Fat_Content"))
    ).withColumn("Item_Fat_Content", 
            sf.when(sf.col("Item_Fat_Content").isNotNull(),
                    sf.regexp_replace(sf.col("Item_Fat_Content"), "(?i)Low Fat", "Lf")
            ).otherwise(sf.col("Item_Fat_Content"))
    ).display()


# Type Casting

    from pyspark.sql.functions import col
    from pyspark.sql import functions as sf
    from pyspark.sql.types import StructType, StructField, StringType, FloatType

    # Define the schema of the DataFrame
    schema = StructType([
        StructField("float_column", FloatType(), nullable=True),
        StructField("string_column", StringType(), nullable=True)
    ])

    # Sample data
    data = [
        (123.456, "123"),
        (789.012, "456"),
        (None, "789")
    ]

    # Create DataFrame
    df = spark.createDataFrame(data, schema=schema)

    # Display original DataFrame
    print("Original DataFrame:")
    df.show()

    # Example of casting a float column to string
    df = df.withColumn('string_from_float', col('float_column').cast('string'))

    # Example of casting a string column to integer
    df = df.withColumn('integer_from_string', col('string_column').cast('integer'))

    # Example of modifiying the type of an existing column by casting
    df = df.withColumn(sf.col("Item_Weight"), sf.col('Item_Weight').cast(sf.StringType()))

    # Display DataFrame after casting
    print("DataFrame after Casting:")
    df.show()

In [0]:
from pyspark.sql import functions as sf
from pyspark.sql.types import DoubleType, StringType

df = df.withColumn('Item_Weight', sf.col('Item_Weight').cast(StringType())) # Using StringType() function to cast the column to string
display(df)

df = df.withColumn('Item_Weight', sf.col('Item_Weight').cast(DoubleType()))
display(df)

# SORT/ORDER BY

      df.sort(sf.col('col_name').asc()) # Sorting a column in ascending order

      df.sort(sf.col('col_name').desc()) # Sorting a column in descending order

      df.sort([sf.col('col_name_1'), sf.col('col_name_2')], ascending = [0,0]) # Sorting both the columns in descending example of 2 stage sorting

      df.sort([sf.col('col_name_1'), sf.col('col_name_2')], ascending = [0,1]) # Sorting first column descending and second ascending example of 2 stage sorting

In [0]:
from pyspark.sql import functions as sf

df.sort(sf.col('Item_Weight').desc()).display()
df.sort(sf.col('Item_Weight').asc()).display()
df.sort([sf.col('Item_Weight'), sf.col('Item_MRP')], ascending = [0,0]).display()
df.sort([sf.col('Item_Weight'), sf.col('Item_MRP')], ascending = [0,1]).display()
df.orderBy(sf.col('Item_Weight').desc(), sf.col('Item_MRP').desc()).display()

# LIMIT



In [0]:
df.limit(20).display()

# Drop a column

In [0]:
df.drop(sf.col("Item_Identifier")).display() # Drop single column
df.drop(sf.col("Item_Identifier"), sf.col("Item_Weight")).display() # Drop multiple columns


# Drop_Duplicates

In [0]:
df.dropDuplicates().display() # Drop duplicate rows
df.dropDuplicates(["Item_Identifier"]).display()
df.dropDuplicates(subset=["Item_Identifier"]).display()
df.distinct().display()